# Desafio

Seu objetivo é identificar quais máquinas apresentam potencial de falha tendo como base dados extraídos através de sensores durante o processo de manufatura.  Para isso são fornecidos dois datasets: um dataset chamado desafio_manutencao_preditiva_treino composto por 6667 linhas, 9 colunas de informação (features) e a variável a ser prevista (“failure_type”).

Participante: Victor Afonso Bauler

# Sumário Executivo



# Solução

## 1 - Importando bibliotecas necessárias

Esta etapa consiste em importar as bibliotecas utilizadas no projeto.

In [44]:
import pandas as pd
import os 

## 2 - Carregamentos dos dados e checagem preliminar

Durante esta etapa é realizada a leitura dos datasets fornecidos e checagem de valores faltantes e duplicados, buscando verificar se existe algum tipo de problema nos dados fornecidos.

In [21]:
# Definindo o diretório raiz
ROOT_DIR = os.path.dirname(os.path.abspath(''))

# Carregando os dados
df_train = pd.read_csv(f'{ROOT_DIR}/data/desafio_manutencao_preditiva_treino.csv')
df_test = pd.read_csv(f'{ROOT_DIR}/data/desafio_manutencao_preditiva_teste.csv')

# Verificando a quantidade de linhas e colunas
print(df_train.shape, df_test.shape)

(6667, 9) (3333, 8)


In [40]:
# Analisando dados faltantes
print(f"Dados faltantes no dataset de treino \n{df_train.isna().sum()}\n")
print(f"Dados faltantes no dataset de teste \n{df_test.isna().sum()}")

Dados faltantes no dataset de treino 
udi                      0
product_id               0
type                     0
air_temperature_k        0
process_temperature_k    0
rotational_speed_rpm     0
torque_nm                0
tool_wear_min            0
failure_type             0
dtype: int64

Dados faltantes no dataset de teste 
udi                      0
product_id               0
type                     0
air_temperature_k        0
process_temperature_k    0
rotational_speed_rpm     0
torque_nm                0
tool_wear_min            0
dtype: int64


In [43]:
# Analisando dados duplicados
print(f"Dados duplicados no dataset de treino \n{df_train.duplicated().sum()}\n")
print(f"Dados duplicados no dataset de teste \n{df_test.duplicated().sum()}")

Dados duplicados no dataset de treino 
0

Dados duplicados no dataset de teste 
0


## 3 - Análise Exploratória

Esta etapa é responsável

## 4 - Modelo Baseline

## 5 - Otimização de hiperparâmetros

## 6 - Resultados